# CSC402 Final Project
## Timothy Colaneri (100652032)
### Fall 2019

## Project: Vintage Basic interpreter written in Python

## Introduction
Initailly, I was pretty unfamiliar with Vintage, or Altair, BASIC. I had taken a few database courses in the past in which I had the opportunity to see a few examples of code written in a few BASIC dialects, so I was not completely foreign to this language. However, I certainly did not have a expert understanding of how its syntax and semantics worked. Below is an example of a program written in this language, taken from the Vintage BASIC homepage's game section. It is a text verion of the game Russian Roulette.

In [1]:
from VintageBASIC_interp import interp

In [2]:
progOne = \
'''
01 PRINT "RUSSIAN ROULETTE"
02 PRINT "CREATIVE COMPUTING   MORRISTOWN, NEW JERSEY"
03 PRINT " " : PRINT " " : PRINT " "
04 PRINT "This is a game of >>>>>>>>> Russian Roulette."
10 PRINT " " : PRINT "Here is a revolver."
20 PRINT "Type '1' to spin the chamber and pull the trigger."
22 PRINT "Type '2' to give up."
23 PRINT "GO"
25 LET n = 0
30 INPUT ; i
31 IF i <> 2 THEN 35
32 PRINT "       Chicken!!!!!"
33 GOTO 99
35 LET n = n + 1
40 IF RND(1) > 0.83333 THEN 70
45 IF n > 10 THEN 80
50 PRINT " - Click - "
60 PRINT " " : GOTO 30
70 PRINT "     BANG!!!!!! You're dead!"
71 PRINT "Condolences will be sent to your relatives."
72 PRINT " " : PRINT " " : PRINT " "
75 PRINT "...next victim..." : GOTO 20
80 PRINT "You Win!!!!!"
85 PRINT "Let someone else blow thier brains out."
90 GOTO 10
99 END
'''
interp(progOne)

RUSSIAN ROULETTE
CREATIVE COMPUTING   MORRISTOWN, NEW JERSEY
 
 
 
This is a game of >>>>>>>>> Russian Roulette.
 
Here is a revolver.
Type '1' to spin the chamber and pull the trigger.
Type '2' to give up.
GO
1
 - Click - 
 
1
     BANG!!!!!! You're dead!
Condolences will be sent to your relatives.
 
 
 
...next victim...
Type '1' to spin the chamber and pull the trigger.
Type '2' to give up.
GO
2
       Chicken!!!!!


I think it makes sense here for me to explain how I handled the labeled lines within this language first as this will flow into and help describe my key data structure for interpreting the program.

One of the first observations I had made after looking at sample code from this langauge is that each line in the program has an associated number attached to it. What I had originally though were probably just lines from the develpment environment turned out to be part of the actual syntax of the program.  
  Example program 1:
     
     10 print "a"
     20 print "b"
     30 print "c"

Each line in Vintage BASIC has a label space at the very beginning of it. These labels serve two purposes. First, they define the order in which the lines of the program should be executed. Second, they provide targets for 'jump' commands, which allow the flow of the program to change to another designated part of the program.  
  Example Program 2: will execute the same order as example 1 due to flow control of program
    
    30 print "c"
    10 print "a"
    20 print "b"

The language that I am interpreting in, Python, has no such flow control labels. Python executes in a traditionaly imperative way, line by line, one at a time. As such, I can't 1:1 map this behavior into Python. I will have to derive my own method of emulating this behavior.

I will need to be able to index into and switch control of my program to a indicated label at any time in the program. One similiar example that we had worked on in class is the stack machine, which had label spaces. In this example we used a list, which can operate like a stack in Python, to allow us to index to a given point in the stack. A list structure would work well for indexing into certain parts of the code. However, each line in the stack machine could only have one command, making its interpreter more simple then ours will have to be, considering we can have multiple statements on a line, serperated by ':'.  
Example Program 3:
    
    10 let y = 10 : print y : let x = 42 : print x

In order to interpret a list of statements, we can use a tree walking structure. Similiar to many of the in class examples, we recursively walk down a tree and execute the different instructions based on the the type of node generated by grammar and parser generator(PLY). Which leads us into..

#### Implementation \ Key Data Structure

By storing each line of code( label space) in a different element in a list, we will be able to index into a given point(label) of the program whenever we encounter a 'jump' command. By keeping a 1:1 parallel list of the actual labels, we will know which statement list belongs to which label.

When we first take code into the interpreter, the very first part of the processing is to 'split' the program into two lists, one of labels and one of the associated code. These lists are parallel.

The quicksort algorithm is then used to place these instructions in execution order which will allow far easier processing in our imperative interpretation environment.

The code is then put back together in execution order and passed into the parser to populate the second list with tuples containing each lines code parallel to the label list.  
Essentially..  
List1 = list of label space names  
List2 = List of label space code  
There is a 1:1 direct parallel realationship between these lists. (ex. List1[0] and List2[0] contain the label name and code of the same label.)  

Each statements implementation is expanded upon in the examples section. 

## Challenges
I had run into quite a few more challenges then I thought I would when implementing this interpreter. I was under the impression that BASIC was a little more simplier then it turned out to be.  
  
  
*** My initial implementation stored the program everytime we encountered a line into a symbol table. The way this worked was that every line's entry in the symbol table contained not only that lines code but every single following lines code. For example, the first line had the entire program stored within its table entry, the second line had the entire program minus the first line... etc..  
This made my required memory space the summation of the space required for the input program. As we know from cominatorics, this is n^2 + n memory space(constant dropped). As we also know from lecture, the possible input set for any(or most) programming languages is infinite, there are lots of possibilities for large programs. I'm glad I was able to think of another implementation!  
  
  

*** Besides, the previously mentioned labels, another issue I had in this language was constructing its for loops. I had to go to office hours to find a way to parse them as my initial implementation had somehow confused that parser generator. Once I had successfully parsed these statements it was pretty downhill in terms of difficulty for implementing it.  
However... I have made an observation about Vintage BASIC for loops that I would like to share.  
When I initially came to office hours to find a way to parse the for loops, one of my grammar ideas looked like this...
    
    stmt : FOR ID '=' EXP opt_step 
         | NEXT op_var
         
We had determined that this could not be the grammar as you would not be able to get all of the code into one tuple, in addition to the NEXT statement being a valid sentence.  
After some research, I believe that this may be the actual implementation of the grammar (or at least something like it), meaning that Vintage BASIC does not have a static for loop structure but rather somewhat of a dynamic one.
This raises many questions as to how such a system would work, take for example this quote from the Vintage BASIC homepage concerning the NEXT statement.  
  
" How does the interpreter determine which FOR goes with the NEXT? It is determined dynamically. If the NEXT has no variable, then it is the most recently encountered non-terminated FOR. If the NEXT indicates a variable, then it is the most recently encountered non-terminated FOR with that control variable. Usually, it is simple to glance at code and see which pairs of FOR and NEXT statements go together. But due to the dynamic nature of the connection between them, it is possible to write NEXT statements that are sometimes connected with one FOR, sometimes another."  
  
As you can see, a NEXT statement may be connected to multiple FOR loops. While I'm not convinced that my above grammar is that actual implementation, I also don't think that the solution we had come to will parse such a structure. The next statement having an optional variable is very telling as well, this statement can encounter a for loop and decide that it is the wrong one to update with this variable attached. In my implementation, below, every FOR loop is connected to its own NEXT statement.
    
    stmt : FOR ID '=' exp opt_step stmts NEXT opt_var
    
*** I also had a little trouble with the RESTORE statement. I believe this problem that I ran into is the same that I had recieved help for during office hours for the FOR loop. There are two forms of RESTORE; RESTORE, and RESTORE label. I believe my grammer for RESTORE label confused the parser generator; I think that whenever it didn't encounter a label, it read the label from the following line, failing the parsing process. The RESTORE only statements works fine. Here is what this looked like in the grammar.
    
    stmt : RESTORE opt_int
    
    opt_int : INTEGER
            | empty

I expand into what I would have done differently in my conclusion at the end.



In [1]:
from VintageBASIC_interp import interp

# Examples
## Flow Control
My implementation for flow control was explained above. The two following examples display the interpreters ability to handle flow control. Both contain the same program, with second one having its order scrambled. They will both output in the same order.

In [2]:
prog = \
'''
10 PRINT "1"
20 PRINT "2"
30 PRINT "3"
40 PRINT "4"
50 PRINT "5"
60 PRINT "6"
70 PRINT "7"
80 PRINT "8"
90 PRINT "9"
100 PRINT "10"
'''
interp(prog)

1
2
3
4
5
6
7
8
9
10


In [3]:
prog = \
'''
50 PRINT "5"
20 PRINT "2"
40 PRINT "4"
60 PRINT "6"
70 PRINT "7"
90 PRINT "9"
10 PRINT "1"
100 PRINT "10"
30 PRINT "3"
80 PRINT "8"
'''
interp(prog)

1
2
3
4
5
6
7
8
9
10


As we can see, the flow control of the program was correctly executed, lets try one more example.

In [4]:
prog = \
'''
10 PRINT "1"
20 PRINT "2"
30 GOTO 80
40 PRINT "5"
50 PRINT "6"
60 PRINT "7"
70 GOTO 110
80 PRINT "3"
90 PRINT "4"
100 GOTO 40
110 PRINT "8"
120 GOTO 140
130 PRINT "-1"
140 PRINT "9"
150 PRINT "10"
'''
interp(prog)

1
2
3
4
5
6
7
8
9
10


We see we can handle the flow of jumps here. Lets try scrambling this program!

In [8]:
prog = \
'''
130 PRINT "-1"
30 GOTO 80
60 PRINT "7"
10 PRINT "1"
80 PRINT "3"
40 PRINT "5"
140 PRINT "9"
120 GOTO 140
50 PRINT "6"
150 PRINT "10"
70 GOTO 110
90 PRINT "4"
110 PRINT "8"
20 PRINT "2"
100 GOTO 40
'''
interp(prog)

1
2
3
4
5
6
7
8
9
10



## Statements

### DATA / READ / RESTORE

My implementation utilizes a queue structure in order to emulate the way the data statement 'holds' memory for the read statement. The queue is part of the state object. A backup queue is also kept and holds the last data statements data for use with the restore statement. 

#### Data Statement
The data statement supplies data for the read statement. Data entered into the data statement is held in memory until a read statement is encountered.
#### Read Statement
The read statement takes data from memory and stores it into its varaibles. If there is no data left to assign, an exception is thrown.
#### Restore Statement
The restore statement restores the data from the last data statement back into memory.

In [9]:
prog = \
'''
05 DATA "abc","def","ghi","jkl"
06 READ e,f,g,h
10 DATA 12 , 34 , 56 , 789
20 READ a,b,c,d
25 PRINT e;f;g;h
30 PRINT a;b;c;d
'''
interp(prog)

abc def ghi jkl
12 34 56 789


Here we can see that the data has been successfully stored in the varaibles through the read and data statements.

In [10]:
prog = \
'''
10 DATA 123 , 456 , 789
20 READ a,b,c
30 PRINT a;b;c
40 RESTORE
50 READ d,e,f
60 PRINT d;e;f
'''
interp(prog)

123 456 789
123 456 789


Here we can see that the restore statement successfully repopulated the data queue, letting us read the data again and store it into different varaibles.

### DEF FN 

This is Vintage BASIC's user defined function declaration statement. I'm pretty happy with how this one turned out. 
The purpose of my implementation of VB having a scopable symbol table is to handle the assignments between formal and actual parameters, making sure we don't accidently overwrite a variable. Here are a few examples:

In [12]:
prog = \
'''
10 DEF FN inc(x) = x + 1
20 LET y = inc(1)
30 PRINT y
'''
interp(prog)

2


Here we can see the function worked correctly.

In [13]:
prog = \
'''
06 LET x = 32
10 DEF FN inc(x) = x + 1
20 LET y = inc(1)
30 PRINT y;x
'''
interp(prog)

2 32


Here we see the purpose of our scopable symbol table, the x variable was never written over.

In [14]:
prog = \
'''
10 DEF FN func (x , y , z) = x * y + SQR (z)
20 LET a = 2
22 LET b = 2
26 LET y = func( a , b , 16) 
30 PRINT y
'''
interp(prog)

8.0


NOTE: sqr() is Vintage BASIC squareroot function  
2 * 2 + sqrt(16) = 4 + 4 = 8   
Our function worked correctly.

### END / STOP
The end and stop statements stop execution of the program. I handled this by throwing a custom exception when one of these statements are encountered.
There can only be one end statement traditionally in Vintage BASIC, found at the end of a program. Stop statements direct the flow of the program to the end statement.  
In my implementation, stop statements are actually constructed as end nodes in the frontend grammar, leaving no difference between the two.

In [15]:
prog = \
'''
10 PRINT "1"
20 PRINT "2"
30 PRINT "3"
35 END
40 PRINT "4"
50 PRINT "5"
'''
interp(prog)

1
2
3


In [16]:
prog = \
'''
10 PRINT "1"
20 PRINT "2"
30 PRINT "3"
35 STOP
40 PRINT "4"
50 PRINT "5"
'''
interp(prog)

1
2
3


As you can see, the stop and end statements stop execution of the program.

### IF THEN
A traditional if statement. If the passed in expression evaluates to true, then we execute the associated code.
We have two forms of an if statement here:
#### IF EXP THEN STMTS
If the passed in expression is not equal to 0, we execute the statements.  
Examples:  
1 - handle basic expression  
2 - handle binary expression  
3 - handle varaible in expression  
4 - handle logical expression  

In [17]:
prog = \
'''
10 IF 1 THEN PRINT "We should see this in the output."
20 IF 0 THEN PRINT "We should NOT see this in the output."
'''
interp(prog)

We should see this in the output.


In [18]:
prog = \
'''
10 IF 13 - 12 THEN PRINT "We should see this in the output."
20 IF 123 * 0 THEN PRINT "We should NOT see this in the output."
'''
interp(prog)

We should see this in the output.


In [19]:
prog = \
'''
05 LET y = 3 - 2
06 DEF FN timesZero(x) = x * 0
07 LET z = timesZero(40) 
10 IF y THEN PRINT "We should see this in the output."
20 IF z THEN PRINT "We should NOT see this in the output."
'''
interp(prog)

We should see this in the output.


In [20]:
prog = \
'''
10 IF 1 > 0 THEN PRINT "We should see this in the output."
20 IF 0 < -1 THEN PRINT "We should NOT see this in the output."
'''
interp(prog)

We should see this in the output.


#### IF EXP THEN LABEL
This if switch form evalutes the expression, if its not 0 then the control of the program switches to the designated label.

In [21]:
prog = \
'''
10 PRINT "a"
15 IF 1 THEN 30
20 PRINT "b"
30 PRINT "c"
'''
interp(prog)

a
c


Here we see the interpreter correctly jumped lines.

In [22]:
prog = \
'''
10 PRINT "a"
15 IF 99 - 33 * 3 THEN 40
20 PRINT "b"
30 PRINT "c"
40 PRINT "d"
60 PRINT "e"
'''
interp(prog)

a
b
c
d
e


Here we see it did not jump when the expression evaluated to false.

### DIM
The dim statement creates an array. It should create an array of varaible size and dimensions based on its passed in values. We didn't get to far into array impementation in class, and I had a little trouble here at first. My current implementation supports arrays of one dimension. It utilizes Python lists. The size space is reserved by printing zero to undeclared elements.  
The first example shows that we can store and retrieve values.  
The second examples shows we can use these variables within our math.

In [23]:
prog = \
'''
10 DIM arr (5)
20 LET arr[3] = 7
30 PRINT arr[3]
40 DATA 1,2,3,4
50 READ arr[0],arr[1],arr[2],arr[4]
60 PRINT arr[0];arr[1];arr[2];arr[4]
'''
interp(prog)

7
1 2 3 4


In [24]:
prog = \
'''
10 DEF FN inc(x) = x + 1
20 DIM arr (3)
30 LET arr[0] = 2
40 LET arr[1] = arr[0] + inc(1)
50 PRINT arr[1]
'''
interp(prog)

4


Here we can see we recieve consistant outputs when using the array object elements within our calculations.

### REM
This is just Vintage BASICs comments.
Nothing within these lines has any effect on the program
The are parsed into the tree structure as they have labels and may be a valid destination of jump commands.

In [25]:
prog = \
'''
10 LET x = 1
20 REM Here we can write our comments
30 PRINT x
'''
interp(prog)

1


### FOR NEXT
I had trouble here, as I had to go to office hours to try and figure out a way to parse this structure. After being able to parse the loops, it was pretty straight forward as we have done a few of these in past assignments. 

In [26]:
prog = \
'''
10 FOR x = 0 TO 20
20 PRINT x
30 NEXT x
'''
interp(prog)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [27]:
prog = \
'''
10 FOR x = 0 TO 20 STEP 2
20 PRINT x
30 NEXT x
'''
interp(prog)

0
2
4
6
8
10
12
14
16
18


In [28]:
prog = \
'''
05 DEF FN dec(x) = x - 1
10 FOR x = 20 TO 0 STEP -2
20 PRINT dec(x)
30 NEXT x
'''
interp(prog)

19
17
15
13
11
9
7
5
3
1


#### FOR loop 1 line
Here we demonstrate the second form a for loop may take in Vintage BASIC. A loop may entirely be placed on one line.

In [35]:
prog = \
'''
10 FOR x = 0 TO 10 STEP 1 : PRINT x : PRINT " *" : NEXT x
'''
interp(prog)

0
 *
1
 *
2
 *
3
 *
4
 *
5
 *
6
 *
7
 *
8
 *
9
 *


Our last test shows us that our second form is correctly interpreted.

### GOTO / GOSUB RETURN
These Vintage BASIC statements change the control flow of the current execution to the designated line. As described in my implementation introduction, I indexed each line in parallel lists, i need only to change the index when I encounter these statements in my interpreter. My implementation of GOSUB jumps to the label and continues the program inside a try/catch that catches an exception thrown by the RETURN statement.
#### GOTO 
The goto statement changes the current line that the program is executing to the desired line.
#### GOSUB RETURN
The gosub statement changes the current line that the program is executing to the desired line, then returns after encountering a return statement.  
  
Here is an example of both:

In [39]:
prog = \
'''
10 GOSUB 50
20 PRINT "C"
30 PRINT "D"
40 END
50 PRINT "A"
60 PRINT "B"
70 RETURN
'''
interp(prog)

A
B
C
D


In [40]:
prog = \
'''
10 GOTO 60
20 PRINT "We never go here"
30 PRINT "A"
40 PRINT "B"
50 RETURN
55 PRINT "We never go here either"
60 GOSUB 30
70 PRINT "C"
80 PRINT "D"
'''
interp(prog)

A
B
C
D


Here we see that the flow control of the program was handled correctly.

### LET
This is the assignment statement in Vintage BASIC. Nothing fancy here, variable's values are stored in the state objects symbol table.

In [41]:
prog = \
'''
10 LET a = 100
20 LET b = "abc"
30 LET c = 25 * 4
40 IF a == c THEN LET b = "def"
50 PRINT a;b;c
'''
interp(prog)

100 def 100


We can see that the ouput is consistant with the variables holding their correct values.   
The actual LET key word is optional, this is shown below.

In [42]:
prog = \
'''
10 x = 12 / 3
20 PRINT x
'''
interp(prog)

4.0


### ON 
This statement evaluates an expression passed into it, the result of the expression is used to index into a list of labels, also passed into it, to determine where the flow control of the program should go to next.  
Nothing fancy in the implementation, the expression is just walked and evaluted, then a GOTO statement to the appropriate label is walked.

#### First form: ON exp GOTO labels
Here are some examples of it working..

In [44]:
prog = \
'''
10 ON 3 GOTO 20 , 30 , 40 , 50 , 60
20 PRINT "a"
30 PRINT "b"
40 PRINT "c"
50 PRINT "d"
60 PRINT "e"
'''
interp(prog)

c
d
e


As our expression was 3, we skip to the third label in the list.

In [45]:
prog = \
'''
01 LET x = 12
10 ON x / 3 GOTO 20 , 30 , 40 , 50 , 60
20 PRINT "a"
30 PRINT "b"
40 PRINT "c"
50 PRINT "d"
60 PRINT "e"
'''
interp(prog)

d
e


Here we see it can properly evaluates expressions as well.

#### Second form: ON exp GOSUB labels
Very similiar to the first form, only now we need to keep an eye out for return statements as we will return flow control to the line following the on statement when we encounter a return statement.

In [46]:
prog = \
'''
10 GOTO 80
20 PRINT "a"
30 PRINT "b"
40 PRINT "c"
50 PRINT "d"
60 PRINT "e"
70 RETURN
80 ON 3 GOSUB 20 , 30 , 40 , 50 , 60
90 PRINT "f"
'''
interp(prog)

c
d
e
f


In [47]:
prog = \
'''
05 LET x = 20
10 GOTO 80
20 PRINT "a"
30 PRINT "b"
40 PRINT "c"
50 PRINT "d"
60 PRINT "e"
70 RETURN
80 ON x / 5 GOSUB 20 , 30 , 40 , 50 , 60
90 PRINT "f"
'''
interp(prog)

d
e
f


## BUILT-IN FUNCTIONS
Below I will run a few tests to show that the built-in functions for the interpreter work.  
#### TRIGONOMETRIC FUNCTIONS  
Some basic examples below, results are returned as floating point values.  
SIN = Sine   
COS = Cosine  
TAN = Tangent  
ATN = ArcTangent

In [48]:
prog = \
'''
10 LET a = SIN(1)
20 LET b = COS(1)
30 LET c = TAN(1)
40 LET d = ATN(1)
50 PRINT a;b;c;d
'''
interp(prog)

0.8414709848078965 0.5403023058681398 1.5574077246549023 0.7853981633974483


By using a calculator to check, we can see that the correct values were output by these functions.

#### STRING FUNCTIONS
Below are some tests of the built in Vintage BASIC functions that work with strings.  
  
  
LEFT = cuts chars off the left side of the string based on input expression  
RIGHT = cuts chars off the right side of the string based on input expression  
MID = cuts chars off the left and right sides of the string based on input expressions  
LEN = returns the length of the passed in string  
SPC = returns an string of empty spaces of length equal to the passed in expression

In [49]:
prog = \
'''
10 LET a = "abcdefghijklmnopqrstuvwxyz"
20 LET b = LEFT( a , 5 )
30 LET c = RIGHT( a , 5 )
40 LET d = MID( a , 5, 5)
50 LET e = LEN(a)
60 LET f = SPC(10)
70 PRINT a : PRINT b : PRINT c : PRINT d : PRINT e
80 PRINT "!" , f , "!"
'''
interp(prog)

abcdefghijklmnopqrstuvwxyz
fghijklmnopqrstuvwxyz
abcdefghijklmnopqrstu
fghijklmnopqrstu
26
!          !


As we can see, these string operators correctly 

#### OTHER MATH FUNCTIONS  
Here we will test out some of the other math functions to show that they work correctly.  
  
  
ABS = returns absolute value of the passed in expression  
EXP = returns the number e raised to the power of the expression  
LOG = returns the logarithm, to base e, of the expression  
SGN = returns -1 if the expression is negative, 0 if it is zero, or 1 if it is positive  
SQR = returns the square root on the passed in expression  
VAL = attempts to read the string expression as a floating-point number.
                                            

In [50]:
prog = \
'''
10 LET a = ABS(-100)
20 LET b = EXP(10)
30 LET c = LOG(10)
40 LET d = SGN(-56)
50 LET e = SGN(10)
60 LET f = SQR(16)
70 LET g = VAL("10") + 10
80 PRINT a : PRINT b : PRINT c : PRINT d : PRINT e : PRINT f : PRINT g
'''
interp(prog)

100
22026.465794806703
2.302585092994046
-1
1
4.0
20.0


As we can see by observing the outputs, these built in functions are operating correctly.

#### OTHER FUNCTIONS  
Here is where I will demonstrate that the other functions which do not fit into the previous 2 catagories work.  
RND(x) = Vintage BASICS Random function:  
if x > 0 then return a random number between 0 and 1  
if x < 0 then reseed the number generator   
if x == 0 return the last random number generated.


In [51]:
prog = \
'''
10 PRINT RND(1)
20 PRINT RND(0)
30 PRINT RND(1)
40 PRINT RND(0)
'''
interp(prog)

0.763774618976614
0.763774618976614
0.2550690257394217
0.2550690257394217


As we can see by the given outputs, the random number generator is working.

#### ASCII FUNCTIONS
ASC = returns then ASCII character value of the first char in the passed in string  
CHR = returns a single-character string with the specified ASCII value  

In [52]:
prog = \
'''
10 LET x = ASC("test")
20 PRINT x
30 LET y = CHR(x)
40 PRINT y
50 LET x = ASC("word")
60 PRINT x
70 LET y = CHR(x)
80 PRINT y
'''
interp(prog)

116
t
119
w


As we can observe by the given outputs of the functions, the ASCII functions are operating correctly.

# Conclusions
I learned a lot through the process of constructing this interpreter, and I have quite a few ideas for improvements. These are some of the better examples. 
  
  
\- First of all, concerning my key data structure of 2 parallel lists, I think that this could be done in a more elegent way in a language that allows the use of pointers. Pointers allow us to index into a specific memory space. Had I had access to pointers when construcing this inerpreter, I imagine I would not have needed to use lists at all. When parsing the program, Each labels location in memory could be indexed instead in a symbol table. This would allow us to iterate through the program and switch flow control within a nested tuple structure by using these pointers as opposed to needing to index into a list.   
  
\- I would really like to find a method of handling for loops that may have a changing structure. In such an implementation, when encountering a next node, we would need to immediatly recurse up the tree until we get to the appropriate for node. Perhaps such a system could be implemented by throwing custom exceptions. Each FOR node would continue the code in a try statement, while each NEXT statement would throw the type of error the FOR was looking for. The FOR node could then check to see if the recieved variabel is the right one, and if not throw it again until it gets to the right FOR node. I think this would be a good challange to implemenet if I had more time.  
  
\- I based my interpreter contruction on the Vintage BASIC homepage documentation. There exist a few minor inconsistancies accross the interpretation in syntax that could be cleaned up to more exactly emulate the interpreted language. 

\- I could also improve upon the typing. There is not much to typing in Vintage BASIC. However, it appears that there is a varaible naming structure; Specifically, string varaibles should end with the $ character. This is something I had overlooked. At current, typing is handled behind the scenes to emulate Vintage BASIC. My implementation handles what you should be able to do and not be able to do concerning types pretty well.
  
Have a nice holiday!